<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/re_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import numpy as np
import pandas as pd
import re

In [73]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
df = pd.read_csv(file_path)#, on_bad_lines='skip')
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)

def extract_numbers(s):
    if pd.isna(s):
        return None
    numbers = re.findall(r'\d+', str(s))
    if numbers:
        return int(numbers[0])
    return None

df_small = df[['Player', 'Bye', 'POS', 'AVG']].head(250).copy()
df_small['Bye'] = df_small['Bye'].apply(extract_numbers)
df_small['Bye'] = df_small['Bye'].fillna(0).astype(int)

df_small.info()
print(df_small.value_counts('POS'))
print(df_small.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  250 non-null    object 
 1   Bye     250 non-null    int64  
 2   POS     250 non-null    object 
 3   AVG     250 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 7.9+ KB
POS
WR     77
RB     66
QB     31
DST    28
TE     28
K      20
Name: count, dtype: int64
                 Player  Bye POS   AVG
0   Christian McCaffrey    9  RB   1.0
1           CeeDee Lamb    7  WR   2.6
2           Tyreek Hill    6  WR   3.2
3        Bijan Robinson   12  RB   5.0
4           Breece Hall   12  RB   5.4
5     Amon-Ra St. Brown    5  WR   6.2
6         Ja'Marr Chase   12  WR   6.6
7      Justin Jefferson    6  WR   7.0
8        Saquon Barkley    5  RB   9.2
9            A.J. Brown    5  WR  10.2
10      Jonathan Taylor   14  RB  10.4
11       Garrett Wilson   12  WR  12.4
12         Jahm

In [74]:
# Anzahl der Wochen
number_of_weeks = 17

# Wochen-Spaltennamen
weekly_columns = [f"Week_{i+1}" for i in range(number_of_weeks)]

# Skalierungsfunktion
def final_projection_base(avg, pos, max_val=22, min_val=7, k=50, c=1.5):
    base = min_val + (max_val - min_val) * (1 / (1 + (avg / k) ** c))
    if pos == "QB":
        base += 4  # QB-Bonus
    return base

final_projections = []
for _, row in df_small.iterrows():
    base_score = final_projection_base(row['AVG'], row['POS'])

    # Erstellen der weekly projections
    weekly_proj = []
    for week in range(number_of_weeks):
        # Überprüfen, ob die aktuelle Woche (week + 1) mit der Bye-Woche des Spielers übereinstimmt
        if (week + 1) == row['Bye']:  # Woche des Spielers = Bye-Woche?
            weekly_proj.append(0.0)  # Projektion auf 0 setzen
        else:
            weekly_proj.append(base_score + np.random.normal(0, base_score * 0.1))  # Zufällige Variation

    final_projections.append(weekly_proj)

# Projektionen in DataFrame einfügen
f = df_small[["Player", "Bye", "POS", "AVG"]].copy()
for i, col in enumerate(weekly_columns):
    f[col] = [proj[i] for proj in final_projections]

# Zeige eine zufällige Stichprobe der ersten 25 Zeilen
f.sample(25)


,Player,Bye,POS,AVG,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,...,Week_8,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17
69,Kyler Murray,11,QB,70.2,16.662796,14.788833,15.873847,13.373149,15.651356,14.340722,...,16.408413,12.645489,14.992995,0.000000,12.020137,18.866087,16.345856,19.612356,14.334023,15.966629
60,Tee Higgins,12,WR,62.2,11.697554,14.148249,12.958100,15.279324,14.275297,12.092951,...,13.211259,13.299069,14.429038,14.823523,0.000000,13.857384,13.566127,15.003887,16.276718,13.926590
130,Trey Benson,11,RB,133.6,11.077459,12.004670,8.750028,11.622808,10.365174,11.077748,...,10.837855,10.102558,8.843383,0.000000,10.440307,9.184089,9.767264,8.009847,8.079504,9.850968
213,Green Bay Packers,10,DST,222.3,8.333655,8.109919,7.241364,8.403092,11.056933,7.987285,...,8.418039,7.939831,0.000000,7.444485,9.211568,8.146895,7.339334,7.017151,8.258566,8.677520
149,Joshua Palmer,12,WR,151.4,6.930441,10.468156,9.446194,9.637737,9.733184,8.484425,...,8.842175,9.355392,9.229907,10.826530,0.000000,10.412180,10.666299,10.138578,10.259259,7.946507
248,Cam Little,12,K,236.7,9.107463,7.735119,7.701904,8.748551,8.590267,7.275841,...,7.595113,7.762157,9.439388,8.471625,0.000000,8.905886,9.568096,9.136382,9.694872,8.044082
187,Jason Sanders,6,K,199.4,8.664970,7.339364,10.038579,8.570174,8.395148,0.000000,...,8.492790,8.182272,7.114960,9.910889,9.510339,7.694465,8.652135,9.269031,9.539817,9.154212
100,Ladd McConkey,5,WR,105.4,11.769068,10.188449,10.117014,11.284892,0.000000,12.169743,...,11.530784,7.692961,10.846060,10.000926,10.096557,9.009069,9.902829,11.420501,9.841596,11.054149
163,Jaylen Wright,6,RB,169.8,8.954007,9.300028,8.420630,7.900590,9.508849,0.000000,...,8.487254,7.749628,8.949524,9.982439,8.609033,10.292432,9.485096,8.169205,9.340320,10.499966
105,Marquise Brown,6,WR,110.2,11.791541,9.993939,10.416487,10.030843,9.678070,0.000000,...,10.044275,9.801141,10.856456,10.553713,13.362343,10.891520,10.763841,11.442428,11.923830,10.139948


In [75]:
week_cols = [col for col in f.columns if col.startswith("Week_")]

# Dictionary: f_dict[player][week] = projection
f_dict = {
    row['Player']: {int(week.replace("Week_", "")): row[week] for week in week_cols}
    for _, row in f.iterrows()
}
list(f_dict.items())[:3]


[('Christian McCaffrey',
  {1: 23.100553082874228,
   2: 22.138022683884746,
   3: 19.479222870741282,
   4: 20.61772876036962,
   5: 19.79550717384562,
   6: 19.294717649201544,
   7: 22.51840238430242,
   8: 18.37829283494255,
   9: 0.0,
   10: 23.123688491864115,
   11: 20.81774202264014,
   12: 18.325802250849378,
   13: 16.94895121224437,
   14: 16.88310223170775,
   15: 16.783179917634033,
   16: 20.051915065001705,
   17: 27.38071471196375}),
 ('CeeDee Lamb',
  {1: 18.348874340773257,
   2: 22.076741298616945,
   3: 23.320153246683837,
   4: 17.491479860432026,
   5: 17.43061548603908,
   6: 23.059188201001856,
   7: 0.0,
   8: 21.51636136789632,
   9: 20.815392481962927,
   10: 22.339525827424136,
   11: 24.63796657604182,
   12: 22.137118468305633,
   13: 19.662729327382916,
   14: 19.967131522776178,
   15: 22.992671433467752,
   16: 21.013007972802964,
   17: 22.89436354288371}),
 ('Tyreek Hill',
  {1: 20.571317631537003,
   2: 20.076974511027103,
   3: 22.58956616040606,
  

In [76]:
week_cols = [col for col in f.columns if col.startswith("Week_")]
f_dict = {
    row['Player']: {int(week.replace("Week_", "")): row[week] for week in week_cols}
    for _, row in f.iterrows()
}
list(f_dict.items())[:3]

[('Christian McCaffrey',
  {1: 23.100553082874228,
   2: 22.138022683884746,
   3: 19.479222870741282,
   4: 20.61772876036962,
   5: 19.79550717384562,
   6: 19.294717649201544,
   7: 22.51840238430242,
   8: 18.37829283494255,
   9: 0.0,
   10: 23.123688491864115,
   11: 20.81774202264014,
   12: 18.325802250849378,
   13: 16.94895121224437,
   14: 16.88310223170775,
   15: 16.783179917634033,
   16: 20.051915065001705,
   17: 27.38071471196375}),
 ('CeeDee Lamb',
  {1: 18.348874340773257,
   2: 22.076741298616945,
   3: 23.320153246683837,
   4: 17.491479860432026,
   5: 17.43061548603908,
   6: 23.059188201001856,
   7: 0.0,
   8: 21.51636136789632,
   9: 20.815392481962927,
   10: 22.339525827424136,
   11: 24.63796657604182,
   12: 22.137118468305633,
   13: 19.662729327382916,
   14: 19.967131522776178,
   15: 22.992671433467752,
   16: 21.013007972802964,
   17: 22.89436354288371}),
 ('Tyreek Hill',
  {1: 20.571317631537003,
   2: 20.076974511027103,
   3: 22.58956616040606,
  

In [77]:
print(f_dict['Christian McCaffrey'][9])
print(f_dict['CeeDee Lamb'][17])


0.0
22.89436354288371


In [78]:
#!pip install mip

In [84]:
import pandas as pd
from mip import Model, BINARY, maximize, xsum

# -- Annahme: df ist bereits geladen und enthält die Spalten wie beschrieben --

# 1. Spieler-Liste und Positionszuordnung
players = df_small['Player'].tolist()
pos = dict(zip(df_small['Player'], df_small['POS']))

# 3. Weitere Parameter
weeks = list(range(1, 18))
positions = {"QB", "RB", "WR", "TE", "K", "DST"}
gamma = {"QB": 2, "RB": 5, "WR": 6, "TE": 2, "K": 1, "DST": 2}
pos_limit = {"QB": 1, "RB": 2, "WR": 2, "TE": 1, "K": 1, "DST": 1}
beta = {t: 100.0 for t in weeks}  # Beispielhafte Gewinnschwelle

# 4. Beispielhafte Listen für bereits gedraftete Spieler (hier leer)
dm_players = []
opp_players = []

# 5. Initialisierung des Modells
model = Model("FantasyFootball", sense=maximize)

# Entscheidungsvariablen
y = {i: model.add_var(name=f"y_{i}", var_type=BINARY) for i in players}
x = {(i, t): model.add_var(name=f"x_{i}_{t}") for i in players for t in weeks}
z = {t: model.add_var(name=f"z_{t}", var_type=BINARY) for t in weeks}

# Zielfunktion
model.objective = (
    1.0 * xsum(f_dict[i][t] * x[i, t] for i in players for t in weeks) +
    100.0 * xsum(z[t] for t in range(1, 16)) +
    150.0 * xsum(z[t] for t in range(16, 18))
)

# Constraints
for j in positions:
    # Mindestanzahl gedrafteter Spieler pro Position (1c)
    model += xsum(y[i] for i in players if pos[i] == j) >= gamma[j]
    # Positionslimits pro Woche (1d)
    for t in weeks:
        model += xsum(x[i, t] for i in players if pos[i] == j) <= pos_limit[j]

model += xsum(y[i] for i in players) <= 18

# Logische Verknüpfung Draft ↔ Aufstellung (1e)
for i in players:
    for t in weeks:
        model += x[i, t] <= y[i]

# Gewinnbedingung (1f)
for t in weeks:
    model += z[t] <= xsum(f_dict[i][t] * x[i, t] for i in players) / beta[t]

# Bereits gedraftete Spieler (1g-1h)
for i in dm_players:
    model += y[i] == 1
for i in opp_players:
    model += y[i] == 0

# Optimierung starten
model.optimize()


<OptimizationStatus.OPTIMAL: 0>

In [85]:
drafted_players = [i for i in players if y[i].x >= 0.99]
print("Dein Team:")
for player in drafted_players:
    print(player)

# Zielfunktionswert (Gesamtpunkte)
print("Zielfunktionswert:", model.objective_value)

Dein Team:
Josh Allen
Mark Andrews
Zamir White
Rhamondre Stevenson
Javonte Williams
Austin Ekeler
Tyjae Spears
Kirk Cousins
Joshua Palmer
Adam Thielen
Brandin Cooks
Josh Downs
Cairo Santos
Atlanta Falcons
Michael Wilson
Jonnu Smith
New England Patriots
Wan'Dale Robinson
Zielfunktionswert: 0.0
